# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from typing import Dict, List

from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import DateType, StringType, FloatType, ByteType, IntegerType, DecimalType

In [2]:
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

### [Airport Code Table](https://datahub.io/core/airport-codes#data)

It provides information about airports around the world.

In [3]:
# Read in the data here
airport_df = spark.read \
    .option("header", True) \
    .csv("raw_data/airport-codes_csv.csv")
print(airport_df.count())
airport_df.limit(5).toPandas()

55075


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"


### [U.S. City Demographic Data](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)

This dataset contains information on the demographics of all US cities with a population greater than 63 000.

In [4]:
#demographic_df = pd.read_csv('us-cities-demographics.csv', sep=';')
demographic_df = spark.read \
    .option("header", True) \
    .options(delimiter=';') \
    .csv("raw_data/us-cities-demographics.csv")

print("Number of rows: ", demographic_df.count())

print("Minimum population: ", demographic_df.select(F.col("Total Population").cast(IntegerType())).sort(F.col("Total Population").asc()).limit(1).toPandas().iloc[0])
print("Maximum population: ", demographic_df.select(F.col("Total Population").cast(IntegerType())).sort(F.col("Total Population").desc()).limit(1).toPandas().iloc[0])

demographic_df.limit(5).toPandas()

Number of rows:  2891
Minimum population:  Total Population    63215
Name: 0, dtype: int32
Maximum population:  Total Population    8550405
Name: 0, dtype: int32


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.6,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


### [I94 Immigration Data](https://www.trade.gov/national-travel-and-tourism-office)

This dataset contains data about immigrants from the US National Tourism and Trade Office. In addition to the actual data, this dataset also comes with a file in which the codes used are described. For a better understanding of the meaning of the names of the columns in this dataset, I recommend the following projects: [project A](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=41a3047f65ae172a11302e9446151d33dcc86033&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f4d6f64696e6777612f446174612d456e67696e656572696e672d43617073746f6e652d50726f6a6563742f343161333034376636356165313732613131333032653934343631353164333364636338363033332f43617073746f6e6525323050726f6a6563742532305375626d697373696f6e2e6970796e62&logged_in=false&nwo=Modingwa%2FData-Engineering-Capstone-Project&path=Capstone+Project+Submission.ipynb&platform=android&repository_id=261688897&repository_type=Repository&version=99) and [project B](https://www.1week4.com/it/machine-learning/udacity-data-engineering-capstone-project/#1.3.2-The-I94-dataset).

In [5]:
imm_df = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
print("Number of rows: ", imm_df.count())
imm_df.limit(5).toPandas()

Number of rows:  3096313


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,...,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


### [World Temperature Data](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data)

This dataset came from Kaggle. It provides historical information about monthly average temperatures in different cities around the world.

In [6]:
temperature_df = spark.read.option("header", True).csv("raw_data/GlobalLandTemperaturesByCity.csv")
print("Number of rows: ", temperature_df.count())
temperature_df.limit(5).toPandas()

Number of rows:  8599212


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.7369999999999999,Århus,Denmark,57.05N,10.33E
1,1743-12-01,None,None,Århus,Denmark,57.05N,10.33E
2,1744-01-01,None,None,Århus,Denmark,57.05N,10.33E
3,1744-02-01,None,None,Århus,Denmark,57.05N,10.33E
4,1744-03-01,None,None,Århus,Denmark,57.05N,10.33E


### [Country Codes](https://countrycode.org/)

This site provides the name and 2-letter code of all countries in the world.

In [7]:
country_df = pd.read_html('https://countrycode.org/')[1]
country_df["ISO CODES"] = country_df["ISO CODES"].apply(lambda value: value.split(" / ")[0])
country_df.drop(columns=["COUNTRY CODE"], inplace=True)
country_df.rename(columns={"COUNTRY": "country", "ISO CODES": "country_code"}, inplace=True)
country_df = spark.createDataFrame(country_df)

print("Number of rows: ", country_df.count())
country_df.limit(5).toPandas()

Number of rows:  240


,country,country_code
0,Afghanistan,AF
1,Albania,AL
2,Algeria,DZ
3,American Samoa,AS
4,Andorra,AD


### [US States Codes](https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=53971)

This site provides the name and 2-letter code of all US states.

In [8]:
us_states_df = pd.read_html('https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=53971')[0]
us_states_df = us_states_df.drop(columns=["Abbreviation", "Code"])
us_states_df = us_states_df.rename(columns={"State": "state", "Alpha code": "state_code"})
us_states_df = spark.createDataFrame(us_states_df)

print("Number of rows: ", us_states_df.count())
us_states_df.limit(5).toPandas()

Number of rows:  51


,state,state_code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


### [Continent Codes](https://www.php.net/manual/en/function.geoip-continent-code-by-name.php)

This site provides the name and 2-letter code of all continents.

In [9]:
continent_df = pd.read_html('https://www.php.net/manual/en/function.geoip-continent-code-by-name.php', keep_default_na=False)[0]
continent_df = spark.createDataFrame(continent_df)

print("Number of rows: ", continent_df.count())
continent_df.limit(5).toPandas()

Number of rows:  7


,Code,Continent name
0,AF,Africa
1,AN,Antarctica
2,AS,Asia
3,EU,Europe
4,NA,North america


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

In [10]:
# Performing cleaning tasks here

dim_country_temperature_evolution_df = temperature_df.dropna()
dim_country_temperature_evolution_df = dim_country_temperature_evolution_df.withColumn("year", F.year("dt")).withColumn("month", F.month("dt"))
dim_country_temperature_evolution_df = dim_country_temperature_evolution_df.groupBy([F.col("Country").alias("country"), "year", "month"]).agg(
    F.avg("AverageTemperature").alias("average_temperature"),
    F.avg("AverageTemperatureUncertainty").alias("average_temperature_uncertainty"),
) 
dim_country_temperature_evolution_df = dim_country_temperature_evolution_df.withColumn('temperature_id', F.monotonically_increasing_id())

print("Number of rows: ", dim_country_temperature_evolution_df.count())
dim_country_temperature_evolution_df.printSchema()
dim_country_temperature_evolution_df.limit(5).toPandas()

Number of rows:  374572
root
 |-- country: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- average_temperature: double (nullable = true)
 |-- average_temperature_uncertainty: double (nullable = true)
 |-- temperature_id: long (nullable = false)



,country,year,month,average_temperature,average_temperature_uncertainty,temperature_id
0,Denmark,1760,11,3.6350,3.9370,0
1,Denmark,1765,3,3.2695,3.2185,1
2,Denmark,1800,6,12.4530,2.0985,2
3,Denmark,1820,4,6.9540,1.4890,3
4,Denmark,1832,4,6.8715,3.9435,4


In [11]:
from pyspark.sql.dataframe import DataFrame

def get_no_of_chars(df_spark: DataFrame) -> pd.DataFrame:
    """Get the minimum and maximum number of characters per string type columns."""
    for column, dtype in df_spark.dtypes:
        if dtype == 'string':
            df_spark = df_spark.withColumn(f"len_{column}", F.length(column))

    min_list = [F.min(column) for column, dtype in df_spark.dtypes if "len_" in column]
    max_list = [F.max(column) for column, dtype in df_spark.dtypes if "len_" in column]
    no_chars_list = min_list + max_list
    return df_spark.select(no_chars_list).limit(len(no_chars_list)).toPandas().T

get_no_of_chars(dim_country_temperature_evolution_df)

,0
min(len_country),4
max(len_country),34


In [31]:
dim_demographic_df = demographic_df.groupBy(F.col("State Code").alias("state_code")).agg(
    F.sum("Total Population").cast(IntegerType()).alias("total_population"),
    F.sum("Male Population").cast(IntegerType()).alias("male_population"),
    F.sum("Female Population").cast(IntegerType()).alias("female_population"),
    F.sum("Number of Veterans").cast(IntegerType()).alias("number_of_veterans"),
    F.sum("Foreign-born").cast(IntegerType()).alias("foregin_born"),
    F.avg("Median Age").cast(FloatType()).alias("median_age"),
    F.avg("Average Household Size").cast(FloatType()).alias("average_household_size"),
) 

print("Number of rows: ", dim_demographic_df.count())
dim_demographic_df.printSchema()
dim_demographic_df.limit(5).toPandas()

Number of rows:  49
root
 |-- state_code: string (nullable = true)
 |-- total_population: integer (nullable = true)
 |-- male_population: integer (nullable = true)
 |-- female_population: integer (nullable = true)
 |-- number_of_veterans: integer (nullable = true)
 |-- foregin_born: integer (nullable = true)
 |-- median_age: float (nullable = true)
 |-- average_household_size: float (nullable = true)



,state_code,total_population,male_population,female_population,number_of_veterans,foregin_born,median_age,average_household_size
0,AZ,22497710,11137275,11360435,1322525,3411565,35.037498,2.774375
1,SC,2586976,1265291,1321685,163334,134019,33.825001,2.469583
2,LA,6502975,3134990,3367985,348855,417095,34.625000,2.465000
3,MN,7044165,3478803,3565362,321738,1069888,35.579628,2.496852
4,NJ,6931024,3423033,3507991,146632,2327750,35.254387,2.960877


In [13]:
get_no_of_chars(dim_demographic_df)

,0
min(len_state_code),2
max(len_state_code),2


In [14]:
from utils.process_data import extract_region
    
dim_world_airports_df = airport_df.where(F.col("type").isin(["small_airport", "medium_airport", "large_airport"]))
dim_world_airports_df = dim_world_airports_df.join(continent_df, F.col("continent") == continent_df.Code, how="left"). \
    join(country_df, F.col("iso_country") == country_df.country_code, how="left"). \
    join(us_states_df, extract_region(F.col("iso_region")) == us_states_df.state_code, how="left")

dim_world_airports_df = dim_world_airports_df.select(
    F.col("ident").alias("airport_id"),
    F.col("name").alias("airport_name"),
    F.split(F.col("type"),"_")[0].alias("airport_type"),
    F.col("iata_code"),
    F.col("local_code").alias("municipality_code"),
    F.col("municipality").alias("municipality"),
    extract_region(F.col("iso_region")).alias("region_code"),
    F.col("state").alias("region"),
    F.col("country_code"),
    F.col("country"),
    F.col("Code").alias("continent_code"),
    F.col("Continent name").alias("continent"),
    F.col("elevation_ft").cast(FloatType()),
    F.split(F.col("coordinates"), ", ")[1].cast(FloatType()).alias("latitude"),
    F.split(F.col("coordinates"), ", ")[0].cast(FloatType()).alias("longitude"),
)

print("Number of rows: ", dim_world_airports_df.count())
dim_world_airports_df.printSchema()
dim_world_airports_df.limit(5).toPandas()

Number of rows:  39142
root
 |-- airport_id: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- airport_type: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- municipality_code: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- region: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent_code: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- elevation_ft: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)



,airport_id,airport_name,airport_type,iata_code,municipality_code,municipality,region_code,region,country_code,country,continent_code,continent,elevation_ft,latitude,longitude
0,DAUB,Biskra Airport,medium,BSK,None,Biskra,07,None,DZ,Algeria,AF,Africa,289.0,34.793301,5.738230
1,MM-0002,Hinthada Airport,small,HEB,None,Hinthada,07,None,MM,Myanmar,AS,Asia,32.0,17.633329,95.466698
2,MM-0004,Myoungmya Airport,small,None,None,Myoungmya,07,None,MM,Myanmar,AS,Asia,85.0,16.570999,94.932297
3,VYPN,Pathein Airport,small,BSX,None,Pathein,07,None,MM,Myanmar,AS,Asia,20.0,16.815201,94.779900
4,DIYO,Yamoussoukro Airport,medium,ASK,None,Yamoussoukro,07,None,CI,Ivory Coast,AF,Africa,699.0,6.903170,-5.365580


In [15]:
get_no_of_chars(dim_world_airports_df)

,0
min(len_airport_id),3
min(len_airport_name),3
min(len_airport_type),5
min(len_iata_code),1
min(len_municipality_code),1
min(len_municipality),2
min(len_region_code),1
min(len_region),4
min(len_country_code),2
min(len_country),4


In [16]:
dim_us_states_df = us_states_df

print("Number of rows: ", dim_us_states_df.count())
dim_us_states_df.printSchema()
dim_us_states_df.limit(5).toPandas()

Number of rows:  51
root
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)



,state,state_code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [17]:
get_no_of_chars(dim_us_states_df)

,0
min(len_state),4
min(len_state_code),2
max(len_state),20
max(len_state_code),2


In [18]:
from utils.read_data import read_i94_descr
input_dir_path = "raw_data"

visa_category = read_i94_descr("I94VISA", input_dir_path)
dim_visa_df = imm_df.select(
    F.col("visatype").alias("visa_type"),
    F.col("visapost").alias("visa_issuer"),
    F.col("I94VISA").alias("visa_category_code").cast(IntegerType()).cast(StringType()),
).dropDuplicates()
dim_visa_df = dim_visa_df.withColumn("visa_category", F.col("visa_category_code")).replace(visa_category, subset="visa_category")

window_visa = Window.orderBy("visa_type")
dim_visa_df = dim_visa_df.withColumn("visa_id", F.row_number().over(window_visa))

print(dim_visa_df.count())
dim_visa_df.printSchema()
dim_visa_df.limit(5).toPandas()

Attribute description I94VISA was successfully read from I94_SAS_Labels_Descriptions.SAS.
2117
root
 |-- visa_type: string (nullable = true)
 |-- visa_issuer: string (nullable = true)
 |-- visa_category_code: string (nullable = true)
 |-- visa_category: string (nullable = true)
 |-- visa_id: integer (nullable = true)



,visa_type,visa_issuer,visa_category_code,visa_category,visa_id
0,B1,VIL,1,Business,1
1,B1,CDJ,1,Business,2
2,B1,SDG,1,Business,3
3,B1,VLL,1,Business,4
4,B1,BOS,1,Business,5


In [19]:
# Even though 'visa_id' is marked as nullable in printSchema(),
# it does not contain Null values
dim_visa_df.where(F.col("visa_id").isNull()).limit(5).toPandas()

,visa_type,visa_issuer,visa_category_code,visa_category,visa_id


In [20]:
get_no_of_chars(dim_visa_df)

,0
min(len_visa_type),1
min(len_visa_issuer),3
min(len_visa_category_code),1
min(len_visa_category),7
max(len_visa_type),3
max(len_visa_issuer),3
max(len_visa_category_code),1
max(len_visa_category),8


In [21]:
from utils.read_data import read_origin_countries

labels = ["origin_country_code", "origin_country"]
values = read_origin_countries(input_dir_path)
dim_applicant_origin_country = spark.createDataFrame(values, labels)

print(dim_applicant_origin_country.count())
dim_applicant_origin_country.printSchema()
dim_applicant_origin_country.limit(5).toPandas()

Attribute description i94res was successfully read from I94_SAS_Labels_Descriptions.SAS.
235
root
 |-- origin_country_code: string (nullable = true)
 |-- origin_country: string (nullable = true)



,origin_country_code,origin_country
0,582,MEXICO
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


In [22]:
get_no_of_chars(dim_applicant_origin_country)

,0
min(len_origin_country_code),3
min(len_origin_country),4
max(len_origin_country_code),3
max(len_origin_country),31


In [23]:
dim_status_flag = imm_df.select(
    F.col("entdepa").alias("arriaval_flag"),
    F.col("entdepd").alias("departure_flag"),
    F.col("entdepu").alias("update_flag"),
    F.col("matflag").alias("match_flag"),
).dropDuplicates()

window_arriaval = Window.orderBy("arriaval_flag")
dim_status_flag = dim_status_flag.withColumn("status_flag_id", F.row_number().over(window_arriaval))

print(dim_status_flag.count())
dim_status_flag.printSchema()
dim_status_flag.limit(5).toPandas()

125
root
 |-- arriaval_flag: string (nullable = true)
 |-- departure_flag: string (nullable = true)
 |-- update_flag: string (nullable = true)
 |-- match_flag: string (nullable = true)
 |-- status_flag_id: integer (nullable = true)



,arriaval_flag,departure_flag,update_flag,match_flag,status_flag_id
0,None,R,None,M,1
1,None,I,None,M,2
2,None,O,None,M,3
3,None,K,None,M,4
4,None,J,None,M,5


In [24]:
get_no_of_chars(dim_status_flag)

,0
min(len_arriaval_flag),1
min(len_departure_flag),1
min(len_update_flag),1
min(len_match_flag),1
max(len_arriaval_flag),1
max(len_departure_flag),1
max(len_update_flag),1
max(len_match_flag),1


In [25]:
labels = ["admission_port_code", "admission_port"]
values = read_i94_descr("i94port", input_dir_path, clean_descr=True).items()
dim_admission_port = spark.createDataFrame(values, labels)

print(dim_admission_port.count())
dim_admission_port.printSchema()
dim_admission_port.limit(5).toPandas()

Attribute description i94port was successfully read from I94_SAS_Labels_Descriptions.SAS.
587
root
 |-- admission_port_code: string (nullable = true)
 |-- admission_port: string (nullable = true)



,admission_port_code,admission_port
0,ALC,"ALCAN, AK"
1,ANC,"ANCHORAGE, AK"
2,BAR,"BAKER AAF - BAKER ISLAND, AK"
3,DAC,"DALTONS CACHE, AK"
4,PIZ,"DEW STATION PT LAY DEW, AK"


In [26]:
get_no_of_chars(dim_admission_port)

,0
min(len_admission_port_code),3
min(len_admission_port),3
max(len_admission_port_code),3
max(len_admission_port),38


In [27]:
mode = read_i94_descr("i94mode", input_dir_path)
dim_arriaval_mode = imm_df.select(
    F.col("i94mode").alias("mode_code").cast(IntegerType()).cast(StringType()),
    F.col("airline"),
    F.col("fltno").alias("flight_number"),
).dropDuplicates()
dim_arriaval_mode = dim_arriaval_mode.withColumn("mode", F.col("mode_code")).replace(mode, subset="mode")

window_arriaval_mode = Window.orderBy("mode_code")
dim_arriaval_mode = dim_arriaval_mode.withColumn("arriaval_mode_id", F.row_number().over(window_arriaval_mode))

print(dim_arriaval_mode.count())
dim_arriaval_mode.printSchema()
dim_arriaval_mode.limit(5).toPandas()

Attribute description i94mode was successfully read from I94_SAS_Labels_Descriptions.SAS.
12020
root
 |-- mode_code: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- arriaval_mode_id: integer (nullable = true)



,mode_code,airline,flight_number,mode,arriaval_mode_id
0,None,None,None,None,1
1,1,RS,07665,Air,2
2,1,LH,00412,Air,3
3,1,DY,7067,Air,4
4,1,3M,00108,Air,5


In [28]:
get_no_of_chars(dim_arriaval_mode)

,0
min(len_mode_code),1
min(len_airline),1
min(len_flight_number),1
min(len_mode),3
max(len_mode_code),1
max(len_airline),3
max(len_flight_number),5
max(len_mode),12


In [29]:
@F.udf(returnType=DateType())
def convert_to_datetime(date: F.col) -> StringType:
    """Convert timestamp to yyyy-mm-dd format."""
    if date is not None:
        return pd.Timestamp("1960-1-1") + pd.to_timedelta(date, unit="D")

admission_date_df = imm_df.select(F.to_date(F.col("dtaddto"),"MMddyyyy").alias("date")).distinct()
added_file_date_df = imm_df.select(F.to_date(F.col("dtadfile"), "yyyyMMdd")).distinct()
arrival_date_df = imm_df.select(convert_to_datetime(F.col("arrdate"))).distinct()
departure_date_df = imm_df.select(convert_to_datetime(F.col("depdate"))).distinct()

date_df = (
        admission_date_df.union(admission_date_df)
        .union(added_file_date_df)
        .union(arrival_date_df)
        .union(departure_date_df)
        .distinct()
    )

date_df = date_df.select(
    F.col("date"),
    F.year(F.col("date")).alias("year"),
    F.quarter(F.col("date")).alias("quarter"),
    F.month(F.col("date")).alias("month"),
    F.weekofyear(F.col("date")).alias("week_of_year"),
    F.dayofweek(F.col("date")).alias("day_of_week"),
    F.dayofmonth(F.col("date")).alias("day_of_month"),
    F.dayofyear(F.col("date")).alias("day_of_year"),
).na.drop(subset=["date"])

print(date_df.count())
date_df.printSchema()
date_df.limit(5).toPandas()

836
root
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- week_of_year: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- day_of_month: integer (nullable = true)
 |-- day_of_year: integer (nullable = true)



,date,year,quarter,month,week_of_year,day_of_week,day_of_month,day_of_year
0,2017-09-11,2017,3,9,37,2,11,254
1,2017-08-11,2017,3,8,32,6,11,223
2,2016-03-01,2016,1,3,9,3,1,61
3,2016-04-25,2016,2,4,17,2,25,116
4,2018-03-17,2018,1,3,11,7,17,76


In [30]:
fact_immigrant_application = imm_df.join(
    F.broadcast(dim_visa_df),
    (imm_df.visatype == dim_visa_df.visa_type)
    & (imm_df.i94visa == dim_visa_df.visa_category_code)
    & (imm_df.visapost == dim_visa_df.visa_issuer),
    how='left'
).join(
    F.broadcast(dim_status_flag),
    (imm_df.entdepa == dim_status_flag.arriaval_flag)
    & (imm_df.entdepd == dim_status_flag.departure_flag)
    & (imm_df.entdepu == dim_status_flag.update_flag)
    & (imm_df.matflag == dim_status_flag.match_flag),
    how='left'
).join(
    dim_arriaval_mode,
    (imm_df.i94mode == dim_arriaval_mode.mode_code)
    & (imm_df.airline == dim_arriaval_mode.airline)
    & (imm_df.fltno == dim_arriaval_mode.flight_number),
    how='left'
)

fact_immigrant_application = fact_immigrant_application.select(
    F.col("cicid").alias("file_id").cast(IntegerType()),
    F.col("insnum").alias("ins_number").cast(IntegerType()),
    F.col("admnum").alias("admission_number").cast(IntegerType()),
    F.col("i94bir").alias("applicant_age").cast(IntegerType()),
    F.col("biryear").alias("applicant_birth_year").cast(IntegerType()),
    F.col("gender"),
    F.col("occup").alias("occupation"),
    F.col("visa_id"),
    F.to_date(F.col("dtadfile"), "yyyyMMdd").alias("application_date"),
    F.col("i94port").alias("admission_port_code"),
    F.col("i94addr").alias("arriaval_state_code"),
    F.col("arriaval_mode_id"),
    convert_to_datetime(F.col("arrdate")).alias("arriaval_date"),
    convert_to_datetime(F.col("depdate")).alias("departure_date"),
    F.to_date(F.col("dtaddto"),"MMddyyyy").alias("limit_date"),
    F.col("status_flag_id"),
    F.col("i94cit").alias("birth_country").cast(IntegerType()).cast(StringType()),
    F.col("i94res").alias("residence_country").cast(IntegerType()).cast(StringType())
)

print(fact_immigrant_application.count())
fact_immigrant_application.printSchema()
fact_immigrant_application.limit(5).toPandas()

3096313
root
 |-- file_id: integer (nullable = true)
 |-- ins_number: integer (nullable = true)
 |-- admission_number: integer (nullable = true)
 |-- applicant_age: integer (nullable = true)
 |-- applicant_birth_year: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- visa_id: integer (nullable = true)
 |-- application_date: date (nullable = true)
 |-- admission_port_code: string (nullable = true)
 |-- arriaval_state_code: string (nullable = true)
 |-- arriaval_mode_id: integer (nullable = true)
 |-- arriaval_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- limit_date: date (nullable = true)
 |-- status_flag_id: integer (nullable = true)
 |-- birth_country: string (nullable = true)
 |-- residence_country: string (nullable = true)



,file_id,ins_number,admission_number,applicant_age,applicant_birth_year,gender,occupation,visa_id,application_date,admission_port_code,arriaval_state_code,arriaval_mode_id,arriaval_date,departure_date,limit_date,status_flag_id,birth_country,residence_country
0,5915334,5055.0,2147483647,92,1924,M,None,422.0,2016-05-10,SDP,CA,NaN,2016-04-23,None,2016-10-21,None,687,687
1,4503220,NaN,2147483647,37,1979,M,None,NaN,2016-04-24,FTL,None,4661.0,2016-04-24,2016-04-27,2016-07-22,None,135,135
2,311835,NaN,2147483647,33,1983,M,None,10.0,2016-04-02,OPF,None,1749.0,2016-04-02,2016-04-06,2016-10-01,None,512,512
3,3189001,NaN,2147483647,33,1983,M,None,391.0,2016-04-17,OPF,None,1749.0,2016-04-17,2016-04-22,2016-10-16,None,512,512
4,4119775,NaN,2147483647,38,1978,M,None,10.0,2016-04-22,LVG,None,1749.0,2016-04-22,2016-04-25,2016-10-21,None,512,512


In [37]:
get_no_of_chars(fact_immigrant_application)

,0
min(len_gender),1
min(len_occupation),3
min(len_admission_port_code),3
min(len_arriaval_state_code),1
min(len_birth_country),3
min(len_residence_country),3
max(len_gender),1
max(len_occupation),3
max(len_admission_port_code),3
max(len_arriaval_state_code),2


### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks 

#### The function for data quality checks is check_data_quality from utils folder. It is used in the process-* functions in etl.py.


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.